<a href="https://colab.research.google.com/github/pawelambroziak/big-data-lab/blob/jupyternotebooks_after_run/notebooks/02_ds_lab_infra_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
REPO_NAME="big-data-lab"
USER_NAME="pawelambroziak" # TODO: replace with your GitHub username

In [11]:
%env REPO_NAME=$REPO_NAME
%env USER_NAME=$USER_NAME

env: REPO_NAME=big-data-lab
env: USER_NAME=pawelambroziak


In [12]:
%%shell
rm -rf $REPO_NAME
git clone https://github.com/$USER_NAME/$REPO_NAME.git
cd $REPO_NAME

Cloning into 'big-data-lab'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 133 (delta 38), reused 12 (delta 12), pack-reused 71 (from 3)
Receiving objects: 100% (133/133), 459.78 KiB | 4.30 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [13]:
%run $REPO_NAME/notebooks/lib/00_terraform_setup.ipynb

env: TERRAFORM_VERSION=1.9.8
env: PATH=/content/bin:/content/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
--2025-01-19 21:05:25--  https://releases.hashicorp.com/terraform/1.9.8/terraform_1.9.8_linux_amd64.zip
Resolving releases.hashicorp.com (releases.hashicorp.com)... 18.154.101.2, 18.154.101.62, 18.154.101.110, ...
Connecting to releases.hashicorp.com (releases.hashicorp.com)|18.154.101.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27041434 (26M) [application/zip]
Saving to: ‘terraform_1.9.8_linux_amd64.zip’

terraform_1.9.8_lin 100%[===================>]  25.79M   131MB/s    in 0.2s    

2025-01-19 21:05:26 (131 MB/s) - ‘terraform_1.9.8_linux_amd64.zip’ saved [27041434/27041434]

Archive:  terraform_1.9.8_linux_amd64.zip
  inflating: bin/LICENSE.txt         
  inflating: bin/terraform           
Terraform v1.9.8
on linux_amd64

Your versio

In [14]:
%run $REPO_NAME/notebooks/conf/00_env_variables.ipynb

env: TF_VAR_ds_semester=2024l
env: TF_VAR_user_id=8001
env: TF_VAR_billing_account=017A30-C0B82F-FC5773
env: TF_VAR_region=europe-west1
env: TF_VAR_project_name=ds-2024l-8001
env: GIT_BRANCH=colab-dev


In [15]:
# Authenticate to the Google account
from google.colab import auth
auth.authenticate_user()

### Set the remote bucket for storing Terraform state


In [16]:
import os
TF_VAR_user_id = os.getenv("TF_VAR_user_id")
TF_VAR_user_id

'8001'

In [17]:
import subprocess

command = f"cd {REPO_NAME}/infra && terraform init -backend-config=bucket=ds-2024l-{TF_VAR_user_id}-state"

result = subprocess.run(command, shell=True, check=True)
print("Komenda zakończona sukcesem", result)

Komenda zakończona sukcesem CompletedProcess(args='cd big-data-lab/infra && terraform init -backend-config=bucket=ds-2024l-8001-state', returncode=0)


### Provision the infrastructure

In [18]:
%%shell
cd $REPO_NAME/infra
terraform apply -auto-approve

Acquiring state lock. This may take a few moments...
module.vpc.module.vpc.module.vpc.google_compute_network.network: Refreshing state... [id=projects/ds-2024l-8001/global/networks/main-vpc]
module.vpc.module.vpc.module.subnets.google_compute_subnetwork.subnetwork["europe-west1/dataproc-01"]: Refreshing state... [id=projects/ds-2024l-8001/regions/europe-west1/subnetworks/dataproc-01]
module.vpc.google_compute_firewall.fw-allow-ingress-from-iap: Refreshing state... [id=projects/ds-2024l-8001/global/firewalls/fw-allow-ingress-iap]
module.vpc.module.cloud-router.google_compute_router.router: Refreshing state... [id=projects/ds-2024l-8001/regions/europe-west1/routers/nat-router]
module.vpc.google_compute_firewall.default-internal-allow-all: Refreshing state... [id=projects/ds-2024l-8001/global/firewalls/default-internal-allow-all]
module.vpc.module.cloud-router.google_compute_router_nat.nats["nat-gateway"]: Refreshing state... [id=ds-2024l-8001/europe-west1/nat-router/nat-gateway]
module.v

### Destroy the infrastructure
After you are done with the lab, you can destroy the infrastructure by running the cell below:

In [ ]:
# %%shell
# cd $REPO_NAME/infra
# terraform destroy -auto-approve